# Creating dimension tables

In [42]:
import pandas as pd
import numpy as np
import re
from variables import * 

In [43]:
# getting the vaccine dimension table from postgres
import psycopg2 as pg
from creds import POSTGRES_USERNAME, POSTGRES_PW
conn = pg.connect(
    dbname='cs689_project',  
    user=POSTGRES_USERNAME,
    password=POSTGRES_PW,
    host='localhost',
    port='5432'
)
cursor = conn.cursor()

## Creating datetime dimension first
SCD0

In [4]:
date_range = pd.date_range(start='2000-01-01', end='2100-12-31', freq='MS')
date_dim = pd.DataFrame({
    'date_id': date_range,
    'year': date_range.year,
    'month': date_range.month
})
date_dim.head()

,date_id,year,month
0,2000-01-01,2000,1
1,2000-02-01,2000,2
2,2000-03-01,2000,3
3,2000-04-01,2000,4
4,2000-05-01,2000,5


In [5]:
try:
    insert_query = '''
    INSERT INTO datetime_dim (Year, Month, date_id)
    VALUES (%s, %s, %s)
    ON CONFLICT (date_id) DO NOTHING;
    '''
    for index, row in date_dim.iterrows():
        cursor.execute(insert_query, (row['year'], row['month'], row['date_id']))
    conn.commit()
    print(f"{len(date_dim)} records inserted into date_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Error occurred: duplicate key value violates unique constraint "datetime_dim_pkey"
DETAIL:  Key (date_id)=(2000-01-01) already exists.



## Handling dimensions from children dataset first

In [44]:
df_children = pd.read_csv('dataset/Vaccination_Coverage_among_Young_Children.csv')
df_children.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,DTaP,≥3 Doses,States/Local Areas,North Dakota,2019,Age,19 Months,93.5,88.0 to 96.6,263.0
1,DTaP,≥3 Doses,States/Local Areas,North Dakota,2018,Age,19 Months,95.2,91.0 to 97.5,293.0
2,DTaP,≥3 Doses,States/Local Areas,North Dakota,2018-2019,Age,19 Months,91.8,88.3 to 94.3,556.0
3,Polio,≥3 Doses,States/Local Areas,North Dakota,2021,Age,19 Months,89.4,81.9 to 94.1,143.0
4,Polio,≥2 Doses,States/Local Areas,North Dakota,2021,Age,5 Months,79.3,69.0 to 86.8,143.0


In [45]:
df_children.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128188 entries, 0 to 128187
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Vaccine                  128188 non-null  object 
 1   Dose                     103127 non-null  object 
 2   Geography Type           128188 non-null  object 
 3   Geography                128188 non-null  object 
 4   Birth Year/Birth Cohort  128188 non-null  object 
 5   Dimension Type           128188 non-null  object 
 6   Dimension                128188 non-null  object 
 7   Estimate (%)             128130 non-null  float64
 8   95% CI (%)               128105 non-null  object 
 9   Sample Size              128130 non-null  float64
dtypes: float64(2), object(8)
memory usage: 9.8+ MB


### Creating vaccine dimension
SCD2

In [46]:
#extracting out the unique vaccines in the dataset
df_children['Vaccine'].unique()

array(['DTaP', 'Polio', 'Hep B', 'PCV', '≥1 Dose Varicella',
       '≥1 Dose MMR', 'Hib', 'Hep A', 'Influenza', 'Combined 7 Series',
       'Rotavirus'], dtype=object)

In [47]:
# since we see that there are vaccines like >=1 dose MMR, we want to check that the dose column has reflected the >=1 dose
df_children[df_children['Vaccine'] == '≥1 Dose MMR'].head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
11,≥1 Dose MMR,NaN,States/Local Areas,North Dakota,2021,Age,19 Months,79.9,69.5 to 87.3,143.0
44,≥1 Dose MMR,NaN,States/Local Areas,North Dakota,2020-2021,Age,35 Months,91.0,85.0 to 95.2,391.0
65,≥1 Dose MMR,NaN,States/Local Areas,North Dakota,2020-2021,Age,19 Months,83.8,77.6 to 88.5,391.0
77,≥1 Dose MMR,NaN,States/Local Areas,North Dakota,2021,Age,13 Months,62.7,51.1 to 73.0,143.0
386,≥1 Dose MMR,NaN,States/Local Areas,North Dakota,2018,Age,13 Months,69.9,61.7 to 77.0,293.0


In [48]:
# we fill in the Dose column
for i, row in df_children.iterrows():
    match = re.search(r'(≥\d+|\d+)', row['Vaccine'])
    if match:
        df_children.at[i, 'Dose'] = str(match.group()) + ' Dose'
    else:
        pass  # Do nothing if no match

In [11]:
# since we see that there are vaccines like >=1 dose MMR, we want to check that the dose column has reflected the >=1 dose
df_children[df_children['Vaccine'] == '≥1 Dose MMR'].head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
11,≥1 Dose MMR,≥1 Dose,States/Local Areas,North Dakota,2021,Age,19 Months,79.9,69.5 to 87.3,143.0
44,≥1 Dose MMR,≥1 Dose,States/Local Areas,North Dakota,2020-2021,Age,35 Months,91.0,85.0 to 95.2,391.0
65,≥1 Dose MMR,≥1 Dose,States/Local Areas,North Dakota,2020-2021,Age,19 Months,83.8,77.6 to 88.5,391.0
77,≥1 Dose MMR,≥1 Dose,States/Local Areas,North Dakota,2021,Age,13 Months,62.7,51.1 to 73.0,143.0
386,≥1 Dose MMR,≥1 Dose,States/Local Areas,North Dakota,2018,Age,13 Months,69.9,61.7 to 77.0,293.0


In [13]:
# since there are nan columns in Dose, we check the vaccine related to it
vaccine_no_dose = df_children['Vaccine'][df_children['Dose'].isna()].unique()
# we see that they are for the influenza and rotavirus
# this is ok since the flu and rotavirus only needs 1 dose
vaccine_no_dose

array(['Influenza', 'Rotavirus'], dtype=object)

In [14]:
# to make things consistent, we can replace nan with '1 Dose only' 
df_children['Dose'] = np.where(pd.isnull(df_children['Dose']), '1 Dose Only', df_children['Dose'])

In [15]:
df_children['Dose'].unique()

array(['≥3 Doses', '≥2 Doses', '≥1 Dose', '≥1 Dose, 2 Day', 'Full Series',
       '1 Dose Only', 'Primary Series', '≥4 Doses', '7 Dose',
       '≥1 Dose, 1 Day', '≥1 Dose, 3 Day (Birth Dose)'], dtype=object)

##### Create dimension table

In [16]:
# we extract out the vaccine_dim first
# start with the original df_children
vaccines = df_children['Vaccine'].unique()
vaccines

array(['DTaP', 'Polio', 'Hep B', 'PCV', '≥1 Dose Varicella',
       '≥1 Dose MMR', 'Hib', 'Hep A', 'Influenza', 'Combined 7 Series',
       'Rotavirus'], dtype=object)

In [17]:
def clean_vaccine(vaccine_array):
    original_vaccines_dim = pd.Series(vaccine_array)
    cleaned_vaccines_dim = original_vaccines_dim.str.replace(r'(≥\d+\s*Dose\s*)', '', regex=True).str.strip()
    vaccines_dim = pd.DataFrame(list(zip(original_vaccines_dim, cleaned_vaccines_dim)))
    vaccines_dim.columns = ['Original Vaccine','Cleaned Vaccine']
    return vaccines_dim

In [18]:
vaccines_dim = clean_vaccine(vaccines)
vaccines_dim = vaccines_dim.reset_index(drop=True)
vaccines_dim['id'] = vaccines_dim.index + 1
display(vaccines_dim)

,Original Vaccine,Cleaned Vaccine,id
0,DTaP,DTaP,1
1,Polio,Polio,2
2,Hep B,Hep B,3
3,PCV,PCV,4
4,≥1 Dose Varicella,Varicella,5
5,≥1 Dose MMR,MMR,6
6,Hib,Hib,7
7,Hep A,Hep A,8
8,Influenza,Influenza,9
9,Combined 7 Series,Combined 7 Series,10


We're taking vaccines as an SCD2 as there are vaccines that get updates with new strains often - namely the influenza vaccine

In [19]:
# using dr.chat to fill in the vaccine description
# starting with 2012
vaccine_strains = [
    'Diphtheria, Tetanus, and Bordetella pertussis (whooping cough)',
    'Poliovirus Types 1, 2, and 3 (inactivated)',
    'Hepatitis B virus (HBV)',
    'Streptococcus pneumoniae — multiple serotypes (e.g., 13, 15, or 20)',
    'Varicella-zoster virus (chickenpox)',
    'Measles virus, Mumps virus, Rubella virus',
    'Haemophilus influenzae type b',
    'Hepatitis A virus',
    'A/California/7/2009, A/Victoria/361/2011 (H3N2), B/Wisconsin/1/2010 (Victoria Lineage)',
    'Includes: DTaP, IPV, MMR, Hib, Hep B, Varicella, PCV',
    'Rotavirus types G1, G2, G3, G4, G9 (Rotarix); G1–G4, G9, G12 (RotaTeq)'
]
vaccine_names = [
    'Daptacel, Infanrix',
    'IPOL',
    'Engerix-B, Recombivax HB',
    'Prevnar 13, Prevnar 15, Prevnar 20',
    'Varivax, ProQuad (MMRV combo)',
    'M-M-R II, ProQuad (MMRV combo)',
    'ActHIB, PedvaxHIB, Hiberix',
    'Havrix, Vaqta',
    'Fluzone, Fluarix, FluMist (live), Flublok',
    'Includes: DTaP, IPV, MMR, Hib, Hep B, Varicella, PCV',
    'Rotarix (monovalent), RotaTeq (pentavalent)']
    
vaccine_description = ['Protects against Diphtheria, Tetanus, and Pertussis (whooping cough)',
 'Protects against Poliomyelitis, a viral disease that can cause paralysis',
 'Protects against Hepatitis B, a liver infection caused by the Hepatitis B virus',
 'Protects against Pneumococcal disease, which can cause pneumonia, meningitis, and bloodstream infections',
 'Protects against chickenpox, caused by the varicella-zoster virus',
 'Protects against Measles, Mumps, and Rubella',
 'Protects against Haemophilus influenzae type b, which can cause meningitis and other serious infections',
 'Protects against Hepatitis A, a liver infection spread through contaminated food or water',
 'Protects against seasonal flu viruses',
 'A combination of 7 recommended childhood vaccines, typically DTaP, Polio, MMR, Hib, Hep B, Varicella, and PCV',
 'Protects against Rotavirus, a virus that causes severe diarrhoea in infants and young children']

vaccines_dim['vaccine_description'] = vaccine_description
vaccines_dim['Vaccine Names'] = vaccine_names
vaccines_dim['Vaccine Strains'] = vaccine_strains
vaccines_dim

,Original Vaccine,Cleaned Vaccine,id,vaccine_description,Vaccine Names,Vaccine Strains
0,DTaP,DTaP,1,"Protects against Diphtheria, Tetanus, and Pert...","Daptacel, Infanrix","Diphtheria, Tetanus, and Bordetella pertussis ..."
1,Polio,Polio,2,"Protects against Poliomyelitis, a viral diseas...",IPOL,"Poliovirus Types 1, 2, and 3 (inactivated)"
2,Hep B,Hep B,3,"Protects against Hepatitis B, a liver infectio...","Engerix-B, Recombivax HB",Hepatitis B virus (HBV)
3,PCV,PCV,4,"Protects against Pneumococcal disease, which c...","Prevnar 13, Prevnar 15, Prevnar 20",Streptococcus pneumoniae — multiple serotypes ...
4,≥1 Dose Varicella,Varicella,5,"Protects against chickenpox, caused by the var...","Varivax, ProQuad (MMRV combo)",Varicella-zoster virus (chickenpox)
5,≥1 Dose MMR,MMR,6,"Protects against Measles, Mumps, and Rubella","M-M-R II, ProQuad (MMRV combo)","Measles virus, Mumps virus, Rubella virus"
6,Hib,Hib,7,Protects against Haemophilus influenzae type b...,"ActHIB, PedvaxHIB, Hiberix",Haemophilus influenzae type b
7,Hep A,Hep A,8,"Protects against Hepatitis A, a liver infectio...","Havrix, Vaqta",Hepatitis A virus
8,Influenza,Influenza,9,Protects against seasonal flu viruses,"Fluzone, Fluarix, FluMist (live), Flublok","A/California/7/2009, A/Victoria/361/2011 (H3N2..."
9,Combined 7 Series,Combined 7 Series,10,A combination of 7 recommended childhood vacci...,"Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell...","Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell..."


In [20]:
import datetime as dt
vaccines_dim['start_datetime'] = pd.to_datetime('2011-09')
vaccines_dim['end_datetime'] = pd.to_datetime('2100-12')

vaccines_dim

,Original Vaccine,Cleaned Vaccine,id,vaccine_description,Vaccine Names,Vaccine Strains,start_datetime,end_datetime
0,DTaP,DTaP,1,"Protects against Diphtheria, Tetanus, and Pert...","Daptacel, Infanrix","Diphtheria, Tetanus, and Bordetella pertussis ...",2011-09-01,2100-12-01
1,Polio,Polio,2,"Protects against Poliomyelitis, a viral diseas...",IPOL,"Poliovirus Types 1, 2, and 3 (inactivated)",2011-09-01,2100-12-01
2,Hep B,Hep B,3,"Protects against Hepatitis B, a liver infectio...","Engerix-B, Recombivax HB",Hepatitis B virus (HBV),2011-09-01,2100-12-01
3,PCV,PCV,4,"Protects against Pneumococcal disease, which c...","Prevnar 13, Prevnar 15, Prevnar 20",Streptococcus pneumoniae — multiple serotypes ...,2011-09-01,2100-12-01
4,≥1 Dose Varicella,Varicella,5,"Protects against chickenpox, caused by the var...","Varivax, ProQuad (MMRV combo)",Varicella-zoster virus (chickenpox),2011-09-01,2100-12-01
5,≥1 Dose MMR,MMR,6,"Protects against Measles, Mumps, and Rubella","M-M-R II, ProQuad (MMRV combo)","Measles virus, Mumps virus, Rubella virus",2011-09-01,2100-12-01
6,Hib,Hib,7,Protects against Haemophilus influenzae type b...,"ActHIB, PedvaxHIB, Hiberix",Haemophilus influenzae type b,2011-09-01,2100-12-01
7,Hep A,Hep A,8,"Protects against Hepatitis A, a liver infectio...","Havrix, Vaqta",Hepatitis A virus,2011-09-01,2100-12-01
8,Influenza,Influenza,9,Protects against seasonal flu viruses,"Fluzone, Fluarix, FluMist (live), Flublok","A/California/7/2009, A/Victoria/361/2011 (H3N2...",2011-09-01,2100-12-01
9,Combined 7 Series,Combined 7 Series,10,A combination of 7 recommended childhood vacci...,"Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell...","Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell...",2011-09-01,2100-12-01


Initial load into vaccine_dim

In [21]:
try:
    insert_query = '''
    INSERT INTO vaccine_dim_demo (cleaned_vaccine, vaccine_description, vaccine_strains, vaccine_names,
    start_date, end_date, current_flag)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    '''
    for index, row in vaccines_dim.iterrows():
        cursor.execute(insert_query, (row['Cleaned Vaccine'], row['vaccine_description'], row['Vaccine Strains'],
        row['Vaccine Names'], row['start_datetime'], row['end_datetime'], 'Y'))
    conn.commit()
    print(f"{len(vaccines_dim)} records inserted into vaccine_dim_demo.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

11 records inserted into vaccine_dim_demo.


Delta load. Updating the influenza vaccine with new strains

In [24]:
def update_vaccine_dim(df_update_vaccine):
    update_query = """
    UPDATE vaccine_dim_demo
    SET end_date     = %s - INTERVAL '1 month',
        current_flag = 'N'
    WHERE cleaned_vaccine = %s
    AND current_flag   = 'Y';
    """

    insert_query = """
    INSERT INTO vaccine_dim_demo
    (cleaned_vaccine,
    vaccine_description,
    vaccine_strains,
    vaccine_names,
    start_date,
    end_date,
    current_flag)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (cleaned_vaccine, start_date) DO NOTHING;
    """
    try:
        for _, row in df_update_vaccine.iterrows():
            cursor.execute(
                update_query,
                (row['start_date'], row['Cleaned Vaccine'])
            )
            cursor.execute(
                insert_query,
                (
                    row['Cleaned Vaccine'],
                    row['Vaccine Description'],
                    row['Vaccine Strain'],
                    row['Vaccine Name'],
                    row['start_date'],
                    row['end_date'],
                    'Y'
                )
            )
        conn.commit()
        print(f"{len(df_update_vaccine)} SCD2 records processed.")

    except Exception as e:
        conn.rollback()
        print("Error occurred:", e)


In [25]:
strains_by_year = {
    2012: [
        "A/California/7/2009",
        "A/Victoria/361/2011 (H3N2)",
        "B/Wisconsin/1/2010",
    ],
    2013: [
        "A/California/7/2009",
        "A/Texas/50/2012 (H3N2)",
        "B/Massachusetts/2/2012",
        "B/Brisbane/60/2008",
    ],
    2014: [
        "A/California/7/2009",
        "A/Texas/50/2012 (H3N2)",
        "B/Massachusetts/2/2012",
        "B/Brisbane/60/2008",
    ],
    2015: [
        "A/California/7/2009",
        "A/Switzerland/9715293/2013 (H3N2)",
        "B/Phuket/3073/2013",
        "B/Brisbane/60/2008",
    ],
    2016: [
        "A/California/7/2009",
        "A/Hong Kong/4801/2014 (H3N2)",
        "B/Brisbane/60/2008",
        "B/Phuket/3073/2013",
    ],
    2017: [
        "A/Michigan/45/2015",
        "A/Hong Kong/4801/2014 (H3N2)",
        "B/Brisbane/60/2008",
        "B/Phuket/3073/2013",
    ],
    2018: [
        "A/Michigan/45/2015",
        "A/Singapore/INFIMH-16-0019/2016 (H3N2)",
        "B/Colorado/06/2017",
        "B/Phuket/3073/2013",
    ],
    2019: [
        "A/Brisbane/02/2018",
        "A/Kansas/14/2017 (H3N2)",
        "B/Colorado/06/2017",
        "B/Phuket/3073/2013",
    ],
    2020: [
        "A/Guangdong-Maonan/SWL1536/2019",
        "A/Hong Kong/2671/2019 (H3N2)",
        "B/Washington/02/2019",
        "B/Phuket/3073/2013",
    ],
    2021: [
        "A/Victoria/2570/2019",
        "A/Cambodia/e0826360/2020 (H3N2)",
        "B/Washington/02/2019",
        "B/Phuket/3073/2013",
    ],
    2022: [
        "A/Victoria/2570/2019",
        "A/Darwin/9/2021 (H3N2)",
        "B/Austria/1359417/2021",
        "B/Phuket/3073/2013",
    ],
    2023: [
        "A/Victoria/4897/2022",
        "A/Darwin/9/2021 (H3N2)",
        "B/Austria/1359417/2021",
        "B/Phuket/3073/2013",
    ],
    2024: [
        "A/Victoria/4897/2022",
        "A/Thailand/8/2022 (H3N2)",
        "B/Austria/1359417/2021",
    ],
}


In [26]:
for year, strains in strains_by_year.items():
    combined = ", ".join(strains)
    vaccine = pd.DataFrame({
        'Vaccine Name':        ['Fluzone, Fluarix, FluMist (live), Flublok'],
        'Cleaned Vaccine':     ['Influenza'],
        'Vaccine Strain':             [ combined ],
        'Vaccine Description': ['Protects against seasonal flu viruses'],
        'start_date':          [ pd.to_datetime(f"{year}-09-01") ],
        'end_date':            [ pd.to_datetime("2100-12-01") ],
    })
    update_vaccine_dim(vaccine)

1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.
1 SCD2 records processed.


### Create dose dimension table
SCD1

In [22]:
# we move on to the dose_dim column
# again, start with df_children
dose_dim = df_children['Dose'].unique()
dose_dim

array(['≥3 Doses', '≥2 Doses', '≥1 Dose', '≥1 Dose, 2 Day', 'Full Series',
       '1 Dose Only', 'Primary Series', '≥4 Doses', '7 Dose',
       '≥1 Dose, 1 Day', '≥1 Dose, 3 Day (Birth Dose)'], dtype=object)

In [23]:
# we can start forming dose_dim into a dataframe
dose_dim = pd.DataFrame(dose_dim)
dose_dim.columns = ['Dose']
dose_dim = dose_dim.reset_index(drop=True)
dose_dim['id'] = dose_dim.index + 1
display(dose_dim)

,Dose,id
0,≥3 Doses,1
1,≥2 Doses,2
2,≥1 Dose,3
3,"≥1 Dose, 2 Day",4
4,Full Series,5
5,1 Dose Only,6
6,Primary Series,7
7,≥4 Doses,8
8,7 Dose,9
9,"≥1 Dose, 1 Day",10


### Create geography hierarchy dimension table
SCD1

In [64]:
# we can move on and handle Geography and Geography type
# we will combine this into one hierarchy table geography_dim
df_children[['Geography', 'Geography Type']]

,Geography,Geography Type
0,North Dakota,States/Local Areas
1,North Dakota,States/Local Areas
2,North Dakota,States/Local Areas
3,North Dakota,States/Local Areas
4,North Dakota,States/Local Areas
...,...,...
128183,New Hampshire,States/Local Areas
128184,New Hampshire,States/Local Areas
128185,New Hampshire,States/Local Areas
128186,New Hampshire,States/Local Areas


In [65]:
# we see that if the geography type id States/Local Areas, it means that its either describing:
# state/territory
# municipality (city/town)
# county
df_children['Geography'][df_children['Geography Type'] == 'States/Local Areas'].unique()

array(['North Dakota', 'North Carolina', 'New Jersey', 'New Mexico',
       'Kansas', 'Nebraska', 'Pennsylvania', 'Oregon', 'Indiana',
       'IL-Rest of state', 'Iowa', 'Texas', 'Utah', 'Vermont',
       'U.S. Virgin Islands', 'Tennessee', 'Alaska', 'Alabama',
       'Oklahoma', 'Georgia', 'West Virginia', 'Wisconsin', 'Ohio',
       'Delaware', 'Connecticut', 'Colorado', 'South Dakota', 'Florida',
       'Kentucky', 'Virginia', 'Washington', 'Arizona', 'South Carolina',
       'Louisiana', 'Maine', 'Minnesota', 'Michigan', 'Hawaii', 'Montana',
       'Missouri', 'District of Columbia', 'Mississippi', 'Wyoming',
       'Maryland', 'Puerto Rico', 'Illinois', 'NY-Rest of state',
       'NY-City of New York', 'PA-Philadelphia', 'New Hampshire',
       'Nevada', 'IL-City of Chicago', 'Idaho', 'TX-City of Houston',
       'New York', 'Rhode Island', 'TX-Rest of state', 'Massachusetts',
       'Guam', 'Arkansas', 'California', 'TX-Travis County',
       'TX-Tarrant County', 'TX-Hidalgo Coun

In [66]:
regions = df_children['Geography'][df_children['Geography Type'] == 'HHS Regions/National'].unique()
regions

array(['United States', 'Region 6', 'Region 5', 'Region 10', 'Region 7',
       'Region 3', 'Region 9', 'Region 1', 'Region 8', 'Region 4',
       'Region 2'], dtype=object)

- we see that counties or municipalities are described like 'state-county/municipality'
- we can map the county/municipality to the state using a dictionary
- we can also map the state to the HHS region
- i created 2 dictionaries mapping state to the 2 letter abbreviation and state to region
- and stored them in variables.py

In [67]:
# we define geography_dim first
# and include the original geography type and geography
geography_dim = pd.DataFrame(columns = ['state_territory', 'municipality', 'county', 'region'])
for region in regions[1:]:
    new_row = pd.DataFrame([{'state_territory': None, 'municipality': None, 'county': None, 'region': region, 'Geography Type': 'HHS Regions/National', 'Geography':region}])
    geography_dim = pd.concat([geography_dim, new_row], ignore_index=True)

geography_dim

,state_territory,municipality,county,region,Geography Type,Geography
0,None,None,None,Region 6,HHS Regions/National,Region 6
1,None,None,None,Region 5,HHS Regions/National,Region 5
2,None,None,None,Region 10,HHS Regions/National,Region 10
3,None,None,None,Region 7,HHS Regions/National,Region 7
4,None,None,None,Region 3,HHS Regions/National,Region 3
5,None,None,None,Region 9,HHS Regions/National,Region 9
6,None,None,None,Region 1,HHS Regions/National,Region 1
7,None,None,None,Region 8,HHS Regions/National,Region 8
8,None,None,None,Region 4,HHS Regions/National,Region 4
9,None,None,None,Region 2,HHS Regions/National,Region 2


In [68]:
# since United States is not a region, we fill it in separately
new_row = pd.DataFrame([{'state_territory': None, 'municipality': None, 'county': None, 'region': None, 'Geography Type': 'HHS Regions/National', 'Geography':'United States'}])
geography_dim = pd.concat([geography_dim, new_row], ignore_index=True)
geography_dim


,state_territory,municipality,county,region,Geography Type,Geography
0,None,None,None,Region 6,HHS Regions/National,Region 6
1,None,None,None,Region 5,HHS Regions/National,Region 5
2,None,None,None,Region 10,HHS Regions/National,Region 10
3,None,None,None,Region 7,HHS Regions/National,Region 7
4,None,None,None,Region 3,HHS Regions/National,Region 3
5,None,None,None,Region 9,HHS Regions/National,Region 9
6,None,None,None,Region 1,HHS Regions/National,Region 1
7,None,None,None,Region 8,HHS Regions/National,Region 8
8,None,None,None,Region 4,HHS Regions/National,Region 4
9,None,None,None,Region 2,HHS Regions/National,Region 2


In [69]:
for i in df_children['Geography'][df_children['Geography Type'] == 'States/Local Areas'].unique():
    matched = False
    if i in state_territory:
        region = state_to_region.get(i)
        new_row = pd.DataFrame([{
            'state_territory': i,
            'municipality': None,
            'county': None,
            'region': 'Region ' + str(region) if region else None,
            'Geography Type': 'States/Local Areas',
            'Geography': i
        }])
        geography_dim = pd.concat([geography_dim, new_row], ignore_index=True)
        matched = True
    elif i[0:2] in state_territory.values():
        # get full state name from code
        state = next((k for k, v in state_territory.items() if v == i[0:2]), None)
        region = state_to_region.get(state)
        if 'County' in i:
            county = i[3:]
            new_row = pd.DataFrame([{
                'state_territory': state,
                'municipality': None,
                'county': county,
                'region': 'Region ' + str(region) if region else None,
                'Geography Type': 'States/Local Areas',
                'Geography': i
            }])
        else:
            municipality = i[3:]
            new_row = pd.DataFrame([{
                'state_territory': state,
                'municipality': municipality,
                'county': None,
                'region': 'Region ' + str(region) if region else None,
                'Geography Type': 'States/Local Areas',
                'Geography': i
            }])
        geography_dim = pd.concat([geography_dim, new_row], ignore_index=True)
        matched = True
    if not matched:
        print(f"Unknown state or local area: {i}")


In [70]:
# we include the running index
geography_dim = pd.DataFrame(geography_dim)
geography_dim.columns = ['state_territory', 'municipality', 'county', 'region', 'Geography Type', 'Geography']
geography_dim = geography_dim.reset_index(drop=True)
geography_dim['id'] = geography_dim.index + 1
display(geography_dim)

,state_territory,municipality,county,region,Geography Type,Geography,id
0,None,None,None,Region 6,HHS Regions/National,Region 6,1
1,None,None,None,Region 5,HHS Regions/National,Region 5,2
2,None,None,None,Region 10,HHS Regions/National,Region 10,3
3,None,None,None,Region 7,HHS Regions/National,Region 7,4
4,None,None,None,Region 3,HHS Regions/National,Region 3,5
...,...,...,...,...,...,...,...
74,Texas,None,Hidalgo County,Region 6,States/Local Areas,TX-Hidalgo County,75
75,Texas,None,El Paso County,Region 6,States/Local Areas,TX-El Paso County,76
76,Texas,None,Dallas County,Region 6,States/Local Areas,TX-Dallas County,77
77,Texas,None,Bexar County,Region 6,States/Local Areas,TX-Bexar County,78


In [71]:
# we see that there are other dimensions that needed to be created
df_children['Dimension Type'].unique()

array(['Age', 'Insurance Coverage', 'Overall', 'Poverty',
       'Race and Ethnicity', 'Urbanicity'], dtype=object)

### Create insurance dimension table
SCD2

In [32]:
# we will cover insurance coverage first
# it might be easier to handle each dimension separately
df_insurance = df_children[df_children['Dimension Type'] == 'Insurance Coverage']
df_insurance.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
81,PCV,≥3 Doses,States/Local Areas,North Dakota,2016-2019,Insurance Coverage,Private Insurance Only,95.8,93.6 to 97.9,695.0
82,Hib,Full Series,States/Local Areas,North Dakota,2016-2019,Insurance Coverage,Private Insurance Only,87.6,83.8 to 91.5,695.0
90,PCV,≥4 Doses,States/Local Areas,North Dakota,2016-2019,Insurance Coverage,Private Insurance Only,90.7,87.7 to 93.6,695.0
96,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Insurance Coverage,Private Insurance Only,72.9,68.6 to 77.2,562.0
97,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Insurance Coverage,Other,52.1,33.1 to 71.1,42.0


In [34]:
insurance_dim = df_insurance['Dimension'].unique()
insurance_dim

array(['Private Insurance Only', 'Other', 'Any Medicaid', 'Uninsured'],
      dtype=object)

In [35]:
# form into a dataframe
insurance_dim = pd.DataFrame(insurance_dim)
insurance_dim.columns = ['Insurance Coverage']
insurance_dim = insurance_dim.reset_index(drop=True)
insurance_dim['id'] = insurance_dim.index + 1
display(insurance_dim)

,Insurance Coverage,id
0,Private Insurance Only,1
1,Other,2
2,Any Medicaid,3
3,Uninsured,4


In [203]:
insurance_dim['Description'] = [None, None, 'DTap, Polio, MMR, Varicella, Hep A, Hep B, PCV, Rotavirus, Influenza, Meningococcal vaccines, RSV', None]
insurance_dim

,Insurance Coverage,Description
0,Uninsured,None
1,Other,None
2,Any Medicaid,"DTap, Polio, MMR, Varicella, Hep A, Hep B, PCV..."
3,Private Insurance Only,None


In [208]:
try:
    insert_query = '''
    UPDATE insurance_dim SET coverage_description = %s WHERE insurance_coverage = %s;
    '''
    for index, row in insurance_dim.iterrows():
        cursor.execute(insert_query, (row['Description'], row['Insurance Coverage']))
    conn.commit()
    print(f"{len(insurance_dim)} records inserted into insurance_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

4 records inserted into insurance_dim.


### Poverty dimension table
SCD3

In [36]:
# we can cover Poverty next
df_poverty = df_children[df_children['Dimension Type'] == 'Poverty']
df_poverty.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
84,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Poverty,<133% FPL,54.8,45.7 to 64.0,155.0
85,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Poverty,133% to <400% FPL,65.2,59.7 to 70.8,408.0
107,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Poverty,>400% FPL,76.5,70.7 to 82.4,267.0
221,Hib,Full Series,States/Local Areas,North Dakota,2014-2017,Poverty,>400% FPL,87.3,82.8 to 91.8,267.0
235,Hib,Full Series,States/Local Areas,North Dakota,2014-2017,Poverty,<133% FPL,80.8,73.7 to 87.8,155.0


In [37]:
poverty_dim = df_poverty['Dimension'].unique()
poverty_dim

array(['<133% FPL', '133% to <400% FPL', '>400% FPL'], dtype=object)

In [38]:
# form into a dataframe
poverty_dim = pd.DataFrame(poverty_dim)
poverty_dim.columns = ['Poverty']
poverty_dim = poverty_dim.reset_index(drop=True)
poverty_dim['id'] = poverty_dim.index + 1
display(poverty_dim)

,Poverty,id
0,<133% FPL,1
1,133% to <400% FPL,2
2,>400% FPL,3


### Race and ethnicity dimension
SCD 0

In [39]:
# race and ethnicity next
df_race = df_children[df_children['Dimension Type'] == 'Race and Ethnicity']
df_race.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
89,Hep A,≥2 Doses,States/Local Areas,North Dakota,2016-2019,Race and Ethnicity,"Other or Multiple Races, Non-Hispanic",63.7,49.5 to 77.9,120.0
91,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Race and Ethnicity,"White, Non-Hispanic",68.4,64.1 to 72.6,637.0
95,Hib,Full Series,States/Local Areas,North Dakota,2016-2019,Race and Ethnicity,"Other or Multiple Races, Non-Hispanic",78.9,69.4 to 88.3,120.0
98,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Race and Ethnicity,Hispanic,53.6,37.2 to 69.9,48.0
99,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Race and Ethnicity,"Other or Multiple Races, Non-Hispanic",65.0,54.5 to 75.4,123.0


In [40]:
# original length
df_race.shape[0]

6946

In [41]:
race_dim = df_race['Dimension'].unique()
race_dim

array(['Other or Multiple Races, Non-Hispanic', 'White, Non-Hispanic',
       'Hispanic', 'Black, Non-Hispanic'], dtype=object)

In [42]:
# form into a dataframe
race_dim = pd.DataFrame(race_dim)
race_dim.columns = ['Race and Ethnicity']
race_dim = race_dim.reset_index(drop=True)
race_dim['id'] = race_dim.index + 1
display(race_dim)

,Race and Ethnicity,id
0,"Other or Multiple Races, Non-Hispanic",1
1,"White, Non-Hispanic",2
2,Hispanic,3
3,"Black, Non-Hispanic",4


### Urbanicity dimension table
SCD0

In [43]:
# urbanicity next
df_urban = df_children[df_children['Dimension Type'] == 'Urbanicity']
df_urban.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
92,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Urbanicity,Living In a MSA Principal City,75.9,70.2 to 81.6,285.0
93,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Urbanicity,Living In a MSA Non-Principal City,67.0,56.9 to 77.1,124.0
94,Influenza,1 Dose Only,States/Local Areas,North Dakota,2014-2017,Urbanicity,Living In a Non-MSA,57.2,51.5 to 62.9,421.0
215,Hib,Full Series,States/Local Areas,North Dakota,2014-2017,Urbanicity,Living In a MSA Non-Principal City,79.4,70.9 to 87.8,124.0
219,Polio,≥3 Doses,States/Local Areas,North Dakota,2014-2017,Urbanicity,Living In a MSA Principal City,94.6,91.5 to 97.8,285.0


In [44]:
urban_dim = df_urban['Dimension'].unique()
urban_dim

array(['Living In a MSA Principal City',
       'Living In a MSA Non-Principal City', 'Living In a Non-MSA'],
      dtype=object)

In [45]:
# form into a dataframe
urban_dim = pd.DataFrame(urban_dim)
urban_dim.columns = ['Urbanicity']
urban_dim = urban_dim.reset_index(drop=True)
urban_dim['id'] = urban_dim.index + 1
display(urban_dim)

,Urbanicity,id
0,Living In a MSA Principal City,1
1,Living In a MSA Non-Principal City,2
2,Living In a Non-MSA,3


## Load dimension tables to DB

In [46]:
# load dose_dim into pgAdmin
# we will be loading in as an SCD0
try:
    insert_query = '''
    INSERT INTO dose_dim(dose, dose_id)
    VALUES (%s, %s);
    '''
    for index, row in dose_dim.iterrows():
        cursor.execute(insert_query, (row['Dose'], row['id']))
    conn.commit()
    print(f"{len(dose_dim)} records inserted into dose_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

11 records inserted into dose_dim.


In [47]:
try:
    insert_query = '''
    INSERT INTO geography_dim (state_territory, municipality, county, region, original_geography_type, original_geography, geography_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    '''
    for index, row in geography_dim.iterrows():
        cursor.execute(insert_query, (row['state_territory'], row['municipality'], row['county'], row['region'], row['Geography Type'], row['Geography'], row['id']))
    conn.commit()
    print(f"{len(geography_dim)} records inserted into geography_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

79 records inserted into geography_dim.


In [48]:
try:
    insert_query = '''
    INSERT INTO race_ethnicity_dim (race_ethnicity, race_ethnicity_id)
    VALUES (%s, %s);
    '''
    for index, row in race_dim.iterrows():
        cursor.execute(insert_query, (row['Race and Ethnicity'], row['id']))
    conn.commit()
    print(f"{len(race_dim)} records inserted into race_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

4 records inserted into race_dim.


In [49]:
try:
    insert_query = '''
    INSERT INTO urbanicity_dim (urbanicity, urbanicity_id)
    VALUES (%s, %s);
    '''
    for index, row in urban_dim.iterrows():
        cursor.execute(insert_query, (row['Urbanicity'], row['id']))
    conn.commit()
    print(f"{len(urban_dim)} records inserted into urban_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

3 records inserted into urban_dim.


In [50]:
# convert to SCD3
poverty_description = ['Household income is less than 133% of the Federal Poverty Level',
'Income is between 133% and just under 400% of the Federal Poverty Level',
'Income is above 400% of the Federal Poverty Level'
]

poverty_dim['previous_poverty_description'] = None
poverty_dim.rename(columns={'Poverty':'current_poverty_status'}, inplace=True)
poverty_dim['current_poverty_description'] = poverty_description

In [51]:
try:
    insert_query = '''
    INSERT INTO poverty_dim (poverty_status, current_poverty_description, previous_poverty_description, poverty_id)
    VALUES (%s, %s, %s, %s);
    '''
    for index, row in poverty_dim.iterrows():
        cursor.execute(insert_query, (row['current_poverty_status'], row['current_poverty_description'], row['previous_poverty_description'], row['id']))
    conn.commit()
    print(f"{len(poverty_dim)} records inserted into poverty_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

3 records inserted into poverty_dim.


In [52]:
# we want to make into an SCD2 table
insurance_dim
insurance_dim['start_date'] = pd.to_datetime('2012-01-01').normalize()
insurance_dim

,Insurance Coverage,id,start_date
0,Private Insurance Only,1,2012-01-01
1,Other,2,2012-01-01
2,Any Medicaid,3,2012-01-01
3,Uninsured,4,2012-01-01


In [53]:
insurance_dim['end_date'] = pd.to_datetime("2100-12-01")
insurance_dim['current_flag'] = 'Y'
insurance_dim

,Insurance Coverage,id,start_date,end_date,current_flag
0,Private Insurance Only,1,2012-01-01,2100-12-01,Y
1,Other,2,2012-01-01,2100-12-01,Y
2,Any Medicaid,3,2012-01-01,2100-12-01,Y
3,Uninsured,4,2012-01-01,2100-12-01,Y


In [54]:
try:
    insert_query = '''
    INSERT INTO insurance_dim (insurance_coverage, start_date, end_date, current_flag)
    VALUES (%s, %s, %s, %s);
    '''
    for index, row in insurance_dim.iterrows():
        cursor.execute(insert_query, (row['Insurance Coverage'],row['start_date'], row['end_date'], row['current_flag']))
    conn.commit()
    print(f"{len(insurance_dim)} records inserted into insurance_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

4 records inserted into insurance_dim.


____

## Handling adolescent dataset

In [27]:
df_adolescents = pd.read_csv('dataset/Vaccination_Coverage_among_Adolescents.csv')
df_adolescents.head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,HPV,"≥1 Dose, Males",States/Local Areas,New York,2023,Age,13-17 Years,81.5,75.2 to 86.5,289.0
1,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2023,Age,13-17 Years,90.2,86.8 to 92.8,559.0
2,Tetanus,≥1 Dose Td or Tdap,States/Local Areas,New York,2023,Age,13-17 Years,93.6,90.9 to 95.5,559.0
3,≥1 Dose MenACWY,NaN,States/Local Areas,New York,2023,Age,13-17 Years,95.3,92.7 to 97.0,559.0
4,HPV,"≥1 Dose, Males and Females",States/Local Areas,New York,2023,Age,13-17 Years,79.4,74.8 to 83.3,559.0


In [28]:
df_adolescents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27565 entries, 0 to 27564
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Vaccine/Sample  27565 non-null  object 
 1   Dose            21650 non-null  object 
 2   Geography Type  27565 non-null  object 
 3   Geography       27565 non-null  object 
 4   Survey Year     27565 non-null  object 
 5   Dimension Type  27565 non-null  object 
 6   Dimension       27565 non-null  object 
 7   Estimate (%)    27553 non-null  float64
 8   95% CI (%)      27551 non-null  object 
 9   Sample Size     27553 non-null  float64
dtypes: float64(2), object(8)
memory usage: 2.1+ MB


### Cleaning vaccine column

In [29]:
df_adolescents['Vaccine/Sample'].unique()

array(['HPV', 'Tetanus', '≥1 Dose MenACWY', '≥3 Doses HepB',
       '≥2 Doses MMR', 'Varicella', '≥2 Doses Hep A'], dtype=object)

In [30]:
df_adolescents['Dose'].unique()

array(['≥1 Dose, Males', '≥1 Dose Tdap', '≥1 Dose Td or Tdap', nan,
       '≥1 Dose, Males and Females', 'Up-to-Date, Males',
       '≥1 Dose, Females', 'Up-to-Date, Males and Females',
       '≥2 Doses or history of disease',
       '≥2 Doses with no disease history',
       '≥1 Dose with no disease history', 'History of disease',
       'Up-to-Date, Females', '≥2 Doses, Females', '≥3 Doses, Females',
       '≥3 Doses, Males and Females', '≥2 Doses, Males and Females',
       '≥2 Doses, Males', '≥3 Doses, Males',
       'Series Completion (3 Dose) Among HPV Vaccination Initiators, Females'],
      dtype=object)

In [31]:
# seeing that the Dose information for HepA, HepB, MenACWY, and MMR are NaN, we fill them in
df_adolescents[df_adolescents['Vaccine/Sample'] == '≥2 Doses Hep A']

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
14,≥2 Doses Hep A,NaN,States/Local Areas,New York,2023,Age,13-17 Years,89.0,85.5 to 91.8,559.0
228,≥2 Doses Hep A,NaN,States/Local Areas,New Hampshire,2023,Age,13-17 Years,89.3,83.7 to 93.1,273.0
526,≥2 Doses Hep A,NaN,States/Local Areas,New Hampshire,2022,Age,13-17 Years,86.7,80.9 to 90.9,225.0
527,≥2 Doses Hep A,NaN,States/Local Areas,New Hampshire,2021,Age,13-17 Years,84.1,78.5 to 88.5,282.0
532,≥2 Doses Hep A,NaN,States/Local Areas,New Hampshire,2020,Age,13-17 Years,77.8,72.6 to 82.2,381.0
...,...,...,...,...,...,...,...,...,...,...
26913,≥2 Doses Hep A,NaN,States/Local Areas,North Dakota,2022,Age,13-17 Years,94.5,89.2 to 97.3,280.0
27093,≥2 Doses Hep A,NaN,States/Local Areas,IL-Rest of state,2023,Age,13-17 Years,80.5,75.7 to 84.6,460.0
27433,≥2 Doses Hep A,NaN,States/Local Areas,TX-Bexar County,2020,Age,13-17 Years,87.3,82.4 to 90.9,314.0
27443,≥2 Doses Hep A,NaN,States/Local Areas,TX-Bexar County,2021,Age,13-17 Years,86.5,80.4 to 90.9,296.0


In [32]:
# we fill in the Dose column
for i, row in df_adolescents.iterrows():
    match = re.search(r'(≥\d+|\d+)', row['Vaccine/Sample'])
    if match:
        df_adolescents.at[i, 'Dose'] = str(match.group()) + ' Dose'
    else:
        pass

In [33]:
df_adolescents[df_adolescents['Vaccine/Sample'] == '≥2 Doses Hep A']

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
14,≥2 Doses Hep A,≥2 Dose,States/Local Areas,New York,2023,Age,13-17 Years,89.0,85.5 to 91.8,559.0
228,≥2 Doses Hep A,≥2 Dose,States/Local Areas,New Hampshire,2023,Age,13-17 Years,89.3,83.7 to 93.1,273.0
526,≥2 Doses Hep A,≥2 Dose,States/Local Areas,New Hampshire,2022,Age,13-17 Years,86.7,80.9 to 90.9,225.0
527,≥2 Doses Hep A,≥2 Dose,States/Local Areas,New Hampshire,2021,Age,13-17 Years,84.1,78.5 to 88.5,282.0
532,≥2 Doses Hep A,≥2 Dose,States/Local Areas,New Hampshire,2020,Age,13-17 Years,77.8,72.6 to 82.2,381.0
...,...,...,...,...,...,...,...,...,...,...
26913,≥2 Doses Hep A,≥2 Dose,States/Local Areas,North Dakota,2022,Age,13-17 Years,94.5,89.2 to 97.3,280.0
27093,≥2 Doses Hep A,≥2 Dose,States/Local Areas,IL-Rest of state,2023,Age,13-17 Years,80.5,75.7 to 84.6,460.0
27433,≥2 Doses Hep A,≥2 Dose,States/Local Areas,TX-Bexar County,2020,Age,13-17 Years,87.3,82.4 to 90.9,314.0
27443,≥2 Doses Hep A,≥2 Dose,States/Local Areas,TX-Bexar County,2021,Age,13-17 Years,86.5,80.4 to 90.9,296.0


In [34]:
df_adolescents['Vaccine/Sample'][df_adolescents['Dose']=='≥3 Doses, Males and Females'].unique()

array(['HPV'], dtype=object)

In [35]:
# since the HPV dose has additional male and female dimensions, we can denormalie it further into a new gender dimension
gender = ['Males', 'Females', 'Males and Females']
def extract_gender(dose):
    # Check if there is a comma in the dose string
    if ',' in dose:
        # Extract the portion after the last comma and remove any extra whitespace.
        gender_part = dose.split(',')[-1].strip()
    else:
        gender_part = dose.strip()
        
    # Perform an exact match against our gender list
    if gender_part in gender:
        return gender_part
    else:
        return None
df_adolescents['Gender'] = df_adolescents['Dose'].apply(extract_gender)

In [40]:
df_adolescents[['Dose', 'Gender']][df_adolescents['Vaccine/Sample'] == 'HPV'].head()

,Dose,Gender
0,"≥1 Dose, Males",Males
4,"≥1 Dose, Males and Females",Males and Females
5,"Up-to-Date, Males",Males
6,"≥1 Dose, Females",Females
7,"Up-to-Date, Males and Females",Males and Females


In [189]:
# now that we have denormalised and added a new gender column, we can clean up the Dose column
df_adolescents['Dose'] = df_adolescents['Dose'].apply(
    lambda dose: dose.split(',')[0].strip() if ',' in dose else dose.strip()
)
df_adolescents['Dose'].unique()

array(['≥1 Dose', '≥1 Dose Tdap', '≥1 Dose Td or Tdap', 'Up-to-Date',
       '≥3 Dose', '≥2 Dose', '≥2 Doses or history of disease',
       '≥2 Doses with no disease history',
       '≥1 Dose with no disease history', 'History of disease',
       '≥2 Doses', '≥3 Doses',
       'Series Completion (3 Dose) Among HPV Vaccination Initiators'],
      dtype=object)

In [190]:
df_adolescents[df_adolescents['Dose'] == '≥1 Dose Tdap'].head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
1,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2023,Age,13-17 Years,90.2,86.8 to 92.8,559.0,None
15,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2023,Age,13-15 Years,90.2,85.9 to 93.3,350.0,None
23,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Uninsured,88.3,68.1 to 96.4,36.0,None
24,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Other,91.0,87.0 to 93.8,365.0,None
25,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Any Medicaid,89.2,86.4 to 91.4,897.0,None


In [191]:
df_adolescents[df_adolescents['Dose'] == 'History of disease'].head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
13,Varicella,History of disease,States/Local Areas,New York,2023,Age,13-17 Years,9.8,7.0 to 13.6,559.0,None
126,Varicella,History of disease,States/Local Areas,NY-City of New York,2018,Age,13-17 Years,12.5,8.3 to 18.4,261.0,None
127,Varicella,History of disease,States/Local Areas,NY-City of New York,2016,Age,13-17 Years,13.4,9.6 to 18.4,291.0,None
128,Varicella,History of disease,States/Local Areas,NY-City of New York,2015,Age,13-17 Years,13.7,10.0 to 18.6,313.0,None
129,Varicella,History of disease,States/Local Areas,NY-City of New York,2009,Age,13-17 Years,51.1,43.5 to 58.7,269.0,None


- i'm going to assume that the Tetanus vaccine here is for Tdap: tetanus, diphtheria & pertussis aka whooping cough (more comprehensive)
- The same for dose marked with 'History of disease' as they are for the Varicella (aka chickenpox) vaccine
- generally, you don't need a varicella vaccine if you've already had chickenpox because your body still has protection but you would have a "history of disease" record if your blood sample tested positive for the virus
- not medical advice, going on general knowledge (ianad!)


In [192]:
adolescents_vaccine = df_adolescents['Vaccine/Sample'].unique()
adolescents_vaccine

array(['HPV', 'Tetanus', '≥1 Dose MenACWY', '≥3 Doses HepB',
       '≥2 Doses MMR', 'Varicella', '≥2 Doses Hep A'], dtype=object)

In [193]:
original_vaccines_dim = pd.Series(adolescents_vaccine)
cleaned_vaccines_dim = original_vaccines_dim.str.replace(r'(≥\d+\s*Doses?\s*)', '', regex=True).str.strip()
vaccines_dim = pd.DataFrame(list(zip(original_vaccines_dim, cleaned_vaccines_dim)))
vaccines_dim.columns = ['Original Vaccine','Cleaned Vaccine']
vaccines_dim

,Original Vaccine,Cleaned Vaccine
0,HPV,HPV
1,Tetanus,Tetanus
2,≥1 Dose MenACWY,MenACWY
3,≥3 Doses HepB,HepB
4,≥2 Doses MMR,MMR
5,Varicella,Varicella
6,≥2 Doses Hep A,Hep A


In [194]:
df_adolescents['Dimension'][df_adolescents['Dimension Type'] == 'Age'].unique()

array(['13-17 Years', '13-15 Years'], dtype=object)

In [195]:
# HPV, Tetanus, MenACWY are new
# tetanus in this case would be Tdap (for 13/17 year olds), replace Tetanus with Tdap
vaccines_dim.loc[vaccines_dim['Cleaned Vaccine'] == 'Tetanus', 'Cleaned Vaccine'] = 'Tdap'
vaccines_dim

,Original Vaccine,Cleaned Vaccine
0,HPV,HPV
1,Tetanus,Tdap
2,≥1 Dose MenACWY,MenACWY
3,≥3 Doses HepB,HepB
4,≥2 Doses MMR,MMR
5,Varicella,Varicella
6,≥2 Doses Hep A,Hep A


In [196]:
vaccine_strains = [
    'HPV Types: 6, 11, 16, 18, 31, 33, 45, 52, 58',
    'No strains — protects against the toxin (tetanospasmin) produced by Clostridium tetani',
    'A, C, W, Y (the most common disease-causing strains globally)',
    'Hepatitis B virus (HBV)',
    'Measles virus, Mumps virus, Rubella virus',
    'Varicella-zoster virus (chickenpox)',
    'Hepatitis A virus',
]
vaccine_names = [
    'Gardasil 9',  # HPV
    'Boostrix, Adacel',  # Tdap
    'Menactra, Menveo, MenQuadfi',  # MenACWY
    'Engerix-B, Recombivax HB, Heplisav-B',  # Hepatitis B
    'M-M-R II, ProQuad (MMRV combo)',  # MMR
    'Varivax, ProQuad (MMRV combo)',  # Varicella
    'Havrix, Vaqta'  # Hepatitis A
]
vaccine_description = [
    'Protects against human papillomavirus, which can cause cervical, anal, and other genital cancers, as well as genital warts',  # HPV
    'Protects against Diphtheria, Tetanus, and Pertussis (whooping cough)',  # Tdap
    'Protects against meningococcal disease caused by Neisseria meningitidis, including meningitis and bloodstream infections',  # MenACWY
    'Protects against Hepatitis B, a liver infection caused by the Hepatitis B virus',  # HepB
    'Protects against Measles, Mumps, and Rubella',  # MMR
    'Protects against chickenpox, caused by the varicella-zoster virus',  # Varicella
    'Protects against Hepatitis A, a liver infection spread through contaminated food or water'  # HepA
]
vaccines_dim['Vaccine Description'] = vaccine_description
vaccines_dim['Vaccine Name'] = vaccine_names
vaccines_dim['Vaccine Strain'] = vaccine_strains
vaccines_dim

,Original Vaccine,Cleaned Vaccine,Vaccine Description,Vaccine Name,Vaccine Strain
0,HPV,HPV,"Protects against human papillomavirus, which c...",Gardasil 9,"HPV Types: 6, 11, 16, 18, 31, 33, 45, 52, 58"
1,Tetanus,Tdap,"Protects against Diphtheria, Tetanus, and Pert...","Boostrix, Adacel",No strains — protects against the toxin (tetan...
2,≥1 Dose MenACWY,MenACWY,Protects against meningococcal disease caused ...,"Menactra, Menveo, MenQuadfi","A, C, W, Y (the most common disease-causing st..."
3,≥3 Doses HepB,HepB,"Protects against Hepatitis B, a liver infectio...","Engerix-B, Recombivax HB, Heplisav-B",Hepatitis B virus (HBV)
4,≥2 Doses MMR,MMR,"Protects against Measles, Mumps, and Rubella","M-M-R II, ProQuad (MMRV combo)","Measles virus, Mumps virus, Rubella virus"
5,Varicella,Varicella,"Protects against chickenpox, caused by the var...","Varivax, ProQuad (MMRV combo)",Varicella-zoster virus (chickenpox)
6,≥2 Doses Hep A,Hep A,"Protects against Hepatitis A, a liver infectio...","Havrix, Vaqta",Hepatitis A virus


In [197]:
vaccines_dim['start_date'] = pd.to_datetime('2011-09')
vaccines_dim['end_date'] = pd.to_datetime('2100-12')

vaccines_dim

,Original Vaccine,Cleaned Vaccine,Vaccine Description,Vaccine Name,Vaccine Strain,start_date,end_date
0,HPV,HPV,"Protects against human papillomavirus, which c...",Gardasil 9,"HPV Types: 6, 11, 16, 18, 31, 33, 45, 52, 58",2011-09-01,2100-12-01
1,Tetanus,Tdap,"Protects against Diphtheria, Tetanus, and Pert...","Boostrix, Adacel",No strains — protects against the toxin (tetan...,2011-09-01,2100-12-01
2,≥1 Dose MenACWY,MenACWY,Protects against meningococcal disease caused ...,"Menactra, Menveo, MenQuadfi","A, C, W, Y (the most common disease-causing st...",2011-09-01,2100-12-01
3,≥3 Doses HepB,HepB,"Protects against Hepatitis B, a liver infectio...","Engerix-B, Recombivax HB, Heplisav-B",Hepatitis B virus (HBV),2011-09-01,2100-12-01
4,≥2 Doses MMR,MMR,"Protects against Measles, Mumps, and Rubella","M-M-R II, ProQuad (MMRV combo)","Measles virus, Mumps virus, Rubella virus",2011-09-01,2100-12-01
5,Varicella,Varicella,"Protects against chickenpox, caused by the var...","Varivax, ProQuad (MMRV combo)",Varicella-zoster virus (chickenpox),2011-09-01,2100-12-01
6,≥2 Doses Hep A,Hep A,"Protects against Hepatitis A, a liver infectio...","Havrix, Vaqta",Hepatitis A virus,2011-09-01,2100-12-01


### Delta Load: Vaccine Dim
##### Using the same function update_vacine_dim, update SCD2 vaccine dimension table

In [198]:
update_vaccine_dim(vaccines_dim)

7 SCD2 records processed.


### Delta Load: Dose Dimension

In [77]:
# we move on to the dose_dim column
# again, start with df_children
dose_dim = df_adolescents['Dose'].unique()
dose_dim

array(['≥1 Dose', '≥1 Dose Tdap', '≥1 Dose Td or Tdap', 'Up-to-Date',
       '≥3 Dose', '≥2 Dose', '≥2 Doses or history of disease',
       '≥2 Doses with no disease history',
       '≥1 Dose with no disease history', 'History of disease',
       '≥2 Doses', '≥3 Doses',
       'Series Completion (3 Dose) Among HPV Vaccination Initiators'],
      dtype=object)

In [78]:
# we can start forming dose_dim into a dataframe
dose_dim = pd.DataFrame(dose_dim)
dose_dim.columns = ['Dose']
dose_dim = dose_dim.reset_index(drop=True)
display(dose_dim)

,Dose
0,≥1 Dose
1,≥1 Dose Tdap
2,≥1 Dose Td or Tdap
3,Up-to-Date
4,≥3 Dose
5,≥2 Dose
6,≥2 Doses or history of disease
7,≥2 Doses with no disease history
8,≥1 Dose with no disease history
9,History of disease


In [105]:
# we can move on and handle Geography and Geography type
# we will combine this into one hierarchy table geography_dim
df_adolescents[['Geography', 'Geography Type']]

,Geography,Geography Type
0,New York,States/Local Areas
1,New York,States/Local Areas
2,New York,States/Local Areas
3,New York,States/Local Areas
4,New York,States/Local Areas
...,...,...
27560,TX-Bexar County,States/Local Areas
27561,TX-Bexar County,States/Local Areas
27562,Region 4,HHS Regions/National
27563,Region 4,HHS Regions/National


In [115]:
# we see that if the geography type id States/Local Areas, it means that its either describing:
# state/territory
# municipality (city/town)
# county
df_adolescents['Geography'][df_adolescents['Geography Type'] == 'States/Local Areas'].unique()

array(['New York', 'NY-City of New York', 'New Jersey', 'Mississippi',
       'Kentucky', 'Maine', 'New Hampshire', 'Colorado', 'Nebraska',
       'IL-Rest of state', 'IL-City of Chicago', 'PA-Philadelphia',
       'PA-Rest of state', 'Hawaii', 'U.S. Virgin Islands',
       'TX-Rest of state', 'Minnesota', 'Florida', 'Oklahoma', 'Nevada',
       'Montana', 'Tennessee', 'Missouri', 'Washington', 'Guam', 'Kansas',
       'Alabama', 'Utah', 'California', 'Alaska', 'Delaware', 'Texas',
       'Idaho', 'Virginia', 'Iowa', 'Wyoming', 'Arkansas', 'Georgia',
       'South Dakota', 'Connecticut', 'Oregon', 'Rhode Island',
       'New Mexico', 'Michigan', 'Louisiana', 'West Virginia',
       'Massachusetts', 'District of Columbia', 'TX-City of Houston',
       'TX-Bexar County', 'Pennsylvania', 'North Carolina', 'Puerto Rico',
       'Maryland', 'NY-Rest of state', 'North Dakota', 'TX-Dallas County',
       'TX-El Paso County', 'TX-Tarrant County', 'TX-Hidalgo County',
       'TX-Travis County',

In [116]:
regions = df_adolescents['Geography'][df_adolescents['Geography Type'] == 'HHS Regions/National'].unique()
regions

array(['Region 7', 'Region 1', 'Region 8', 'Region 10', 'Region 6',
       'United States', 'Region 5', 'Region 9', 'Region 4', 'Region 3',
       'Region 2'], dtype=object)

In [117]:
# we define geography_dim first
# and include the original geography type and geography
adolescent_geography_dim = pd.DataFrame(columns = ['state_territory', 'municipality', 'county', 'region'])

In [118]:
df_adolescents[df_adolescents['Geography Type'] == 'HHS Regions/National']

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
858,HPV,"≥1 Dose, Females",HHS Regions/National,Region 7,2011,Age,13-17 Years,49.0,43.7 to 54.3,732.0,Females
859,HPV,"≥1 Dose, Females",HHS Regions/National,Region 7,2010,Age,13-17 Years,44.1,39.4 to 48.8,616.0,Females
860,Varicella,≥2 Doses with no disease history,HHS Regions/National,Region 7,2017,Age,13-17 Years,83.9,80.9 to 86.6,1140.0,None
861,Varicella,≥2 Doses with no disease history,HHS Regions/National,Region 7,2013,Age,13-17 Years,67.4,62.8 to 71.7,858.0,None
862,Varicella,≥2 Doses with no disease history,HHS Regions/National,Region 7,2014,Age,13-17 Years,70.7,66.5 to 74.5,1015.0,None
...,...,...,...,...,...,...,...,...,...,...,...
25612,HPV,"Up-to-Date, Females",HHS Regions/National,Region 2,2020,Age,13-15 Years,59.1,51.7 to 66.2,316.0,Females
25613,HPV,"Up-to-Date, Females",HHS Regions/National,Region 2,2019,Age,13-15 Years,54.2,46.1 to 62.0,254.0,Females
27562,Varicella,History of disease,HHS Regions/National,Region 4,2008,Age,13-17 Years,60.6,57.6 to 63.4,2788.0,None
27563,Varicella,History of disease,HHS Regions/National,Region 4,2006,Age,13-17 Years,69.9,65.0 to 74.4,608.0,None


In [119]:
# we fill in the regions into geography_dim
for region in regions:
    if region != 'United States':
        new_row = pd.DataFrame([{'state_territory': None, 'municipality': None, 'county': None, 'region': region, 'Geography Type': 'HHS Regions/National', 'Geography':region}])
        adolescent_geography_dim = pd.concat([adolescent_geography_dim, new_row], ignore_index=True)

adolescent_geography_dim

,state_territory,municipality,county,region,Geography Type,Geography
0,None,None,None,Region 7,HHS Regions/National,Region 7
1,None,None,None,Region 1,HHS Regions/National,Region 1
2,None,None,None,Region 8,HHS Regions/National,Region 8
3,None,None,None,Region 10,HHS Regions/National,Region 10
4,None,None,None,Region 6,HHS Regions/National,Region 6
5,None,None,None,Region 5,HHS Regions/National,Region 5
6,None,None,None,Region 9,HHS Regions/National,Region 9
7,None,None,None,Region 4,HHS Regions/National,Region 4
8,None,None,None,Region 3,HHS Regions/National,Region 3
9,None,None,None,Region 2,HHS Regions/National,Region 2


In [120]:
# since United States is not a region, we fill it in separately
new_row = pd.DataFrame([{'state_territory': None, 'municipality': None, 'county': None, 'region': None, 'Geography Type': 'HHS Regions/National', 'Geography':'United States'}])
adolescent_geography_dim = pd.concat([adolescent_geography_dim, new_row], ignore_index=True)
adolescent_geography_dim


,state_territory,municipality,county,region,Geography Type,Geography
0,None,None,None,Region 7,HHS Regions/National,Region 7
1,None,None,None,Region 1,HHS Regions/National,Region 1
2,None,None,None,Region 8,HHS Regions/National,Region 8
3,None,None,None,Region 10,HHS Regions/National,Region 10
4,None,None,None,Region 6,HHS Regions/National,Region 6
5,None,None,None,Region 5,HHS Regions/National,Region 5
6,None,None,None,Region 9,HHS Regions/National,Region 9
7,None,None,None,Region 4,HHS Regions/National,Region 4
8,None,None,None,Region 3,HHS Regions/National,Region 3
9,None,None,None,Region 2,HHS Regions/National,Region 2


In [121]:
for i in df_adolescents['Geography'][df_adolescents['Geography Type'] == 'States/Local Areas'].unique():
    matched = False
    if i in state_territory:
        region = state_to_region.get(i)
        new_row = pd.DataFrame([{
            'state_territory': i,
            'municipality': None,
            'county': None,
            'region': 'Region ' + str(region) if region else None,
            'Geography Type': 'States/Local Areas',
            'Geography': i
        }])
        adolescent_geography_dim = pd.concat([adolescent_geography_dim, new_row], ignore_index=True)
        matched = True
    elif i[0:2] in state_territory.values():
        # get full state name from code
        state = next((k for k, v in state_territory.items() if v == i[0:2]), None)
        region = state_to_region.get(state)
        if 'County' in i:
            county = i[3:]
            new_row = pd.DataFrame([{
                'state_territory': state,
                'municipality': None,
                'county': county,
                'region': 'Region ' + str(region) if region else None,
                'Geography Type': 'States/Local Areas',
                'Geography': i
            }])
        else:
            municipality = i[3:]
            new_row = pd.DataFrame([{
                'state_territory': state,
                'municipality': municipality,
                'county': None,
                'region': 'Region ' + str(region) if region else None,
                'Geography Type': 'States/Local Areas',
                'Geography': i
            }])
        adolescent_geography_dim = pd.concat([adolescent_geography_dim, new_row], ignore_index=True)
        matched = True
    if not matched:
        print(f"Unknown state or local area: {i}")


In [122]:
adolescent_geography_dim

,state_territory,municipality,county,region,Geography Type,Geography
0,None,None,None,Region 7,HHS Regions/National,Region 7
1,None,None,None,Region 1,HHS Regions/National,Region 1
2,None,None,None,Region 8,HHS Regions/National,Region 8
3,None,None,None,Region 10,HHS Regions/National,Region 10
4,None,None,None,Region 6,HHS Regions/National,Region 6
...,...,...,...,...,...,...
74,Ohio,None,None,Region 5,States/Local Areas,Ohio
75,Arizona,None,None,Region 9,States/Local Areas,Arizona
76,South Carolina,None,None,Region 4,States/Local Areas,South Carolina
77,Indiana,None,None,Region 5,States/Local Areas,Indiana


In [123]:
len(adolescent_geography_dim)

79

### Creating a new gender dimension table
SCD0

In [199]:
gender_dim = df_adolescents['Gender'].unique()

In [200]:
gender_dim = pd.DataFrame(gender_dim)
gender_dim.columns = ['Gender']
gender_dim = gender_dim.reset_index(drop=True)
gender_dim['id'] = gender_dim.index + 1
display(gender_dim)

,Gender,id
0,Males,1
1,None,2
2,Males and Females,3
3,Females,4


In [201]:
try:
    insert_query = '''
    INSERT INTO gender_dim (
        gender
    )
    VALUES (%s)
    ON CONFLICT (gender) 
    DO UPDATE SET
        gender = EXCLUDED.gender;
    '''
    for index, row in gender_dim.iterrows():
        print(row['Gender'])
        cursor.execute(insert_query, (row['Gender'],))
    conn.commit()
    print(f"{len(gender_dim)} records inserted into gender_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Males
None
Males and Females
Females
4 records inserted into gender_dim.


In [90]:
# we see that there are other dimensions that needed to be created
df_adolescents['Dimension Type'].unique()

array(['Age', 'Insurance Coverage', 'Poverty', 'Race and Ethnicity',
       'Urbanicity', 'Overall'], dtype=object)

### Insurance dimension table

In [91]:
# we will cover insurance coverage first
# it might be easier to handle each dimension separately
df_insurance = df_adolescents[df_adolescents['Dimension Type'] == 'Insurance Coverage']
df_insurance.head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
23,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Uninsured,88.3,68.1 to 96.4,36.0,None
24,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Other,91.0,87.0 to 93.8,365.0,None
25,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Any Medicaid,89.2,86.4 to 91.4,897.0,None
26,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Insurance Coverage,Private Insurance Only,92.6,90.8 to 94.0,1581.0,None
27,HPV,Up-to-Date,States/Local Areas,New York,2018-2022,Insurance Coverage,Uninsured,40.1,23.3 to 59.6,36.0,Males and Females


In [92]:
insurance_dim = df_insurance['Dimension'].unique()
insurance_dim

array(['Uninsured', 'Other', 'Any Medicaid', 'Private Insurance Only'],
      dtype=object)

In [93]:
# form into a dataframe
# use the running serial in the previous insruance dimension table (children)
insurance_dim = pd.DataFrame(insurance_dim)
insurance_dim.columns = ['Insurance Coverage']
insurance_dim = insurance_dim.reset_index(drop=True)
display(insurance_dim)

,Insurance Coverage
0,Uninsured
1,Other
2,Any Medicaid
3,Private Insurance Only


### Poverty dimension table

In [94]:
# we can cover Poverty next
df_poverty = df_adolescents[df_adolescents['Dimension Type'] == 'Poverty']

df_poverty.head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
39,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Poverty,Below Poverty Level,88.8,85.3 to 91.6,533.0,None
40,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Poverty,Living At or Above Poverty Level,91.9,90.4 to 93.2,2180.0,None
41,HPV,Up-to-Date,States/Local Areas,New York,2018-2022,Poverty,Living At or Above Poverty Level,63.0,60.3 to 65.6,2180.0,Males and Females
42,HPV,Up-to-Date,States/Local Areas,New York,2018-2022,Poverty,Below Poverty Level,67.8,62.4 to 72.7,533.0,Males and Females
43,≥1 Dose MenACWY,≥1 Dose,States/Local Areas,New York,2018-2022,Poverty,Living At or Above Poverty Level,95.4,94.1 to 96.3,2180.0,None


In [95]:
poverty_dim = df_poverty['Dimension'].unique()
poverty_dim

array(['Below Poverty Level', 'Living At or Above Poverty Level'],
      dtype=object)

In [109]:
# we note that this is different from the FPL level in children
poverty_dim = pd.DataFrame(poverty_dim)
poverty_dim.columns = ['Poverty']
adolescent_poverty_description = ['Living below poverty level. Undefined percentage.', 'Living above poverty level. Undefined percentage.']
poverty_dim['current_poverty_description'] = adolescent_poverty_description
poverty_dim = poverty_dim.reset_index(drop=True)
display(poverty_dim)

,Poverty,current_poverty_description
0,Below Poverty Level,Living below poverty level. Undefined percentage.
1,Living At or Above Poverty Level,Living above poverty level. Undefined percentage.


### Race and ethnicity dimension

In [96]:
# race and ethnicity next
df_race = df_adolescents[df_adolescents['Dimension Type'] == 'Race and Ethnicity']
df_race.head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
47,HPV,≥1 Dose,States/Local Areas,New York,2018-2022,Race and Ethnicity,"White, Non-Hispanic",70.7,67.4 to 73.8,1333.0,Males and Females
48,HPV,≥1 Dose,States/Local Areas,New York,2018-2022,Race and Ethnicity,"Black, Non-Hispanic",70.4,63.7 to 76.4,322.0,Males and Females
49,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Race and Ethnicity,"Other or Multiple Races, Non-Hispanic",89.6,85.0 to 92.9,435.0,None
50,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Race and Ethnicity,Hispanic,89.7,86.8 to 92.0,789.0,None
51,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Race and Ethnicity,"Black, Non-Hispanic",89.7,85.4 to 92.8,322.0,None


In [97]:
race_dim = df_race['Dimension'].unique()
race_dim

array(['White, Non-Hispanic', 'Black, Non-Hispanic',
       'Other or Multiple Races, Non-Hispanic', 'Hispanic'], dtype=object)

In [98]:
# form into a dataframe
race_dim = pd.DataFrame(race_dim)
race_dim.columns = ['Race and Ethnicity']
race_dim = race_dim.reset_index(drop=True)
display(race_dim)

,Race and Ethnicity
0,"White, Non-Hispanic"
1,"Black, Non-Hispanic"
2,"Other or Multiple Races, Non-Hispanic"
3,Hispanic


### Urbanicity dimension table

In [99]:
df_urban = df_adolescents[df_adolescents['Dimension Type'] == 'Urbanicity']
df_urban.head()

,Vaccine/Sample,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Gender
61,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Urbanicity,Living In a MSA Principal City,89.9,88.0 to 91.5,1728.0,None
62,Tetanus,≥1 Dose Tdap,States/Local Areas,New York,2018-2022,Urbanicity,Living In a MSA Non-Principal City,92.7,90.4 to 94.4,1005.0,None
65,≥1 Dose MenACWY,≥1 Dose,States/Local Areas,New York,2018-2022,Urbanicity,Living In a MSA Principal City,94.7,93.2 to 95.8,1728.0,None
66,≥1 Dose MenACWY,≥1 Dose,States/Local Areas,New York,2018-2022,Urbanicity,Living In a Non-MSA,87.1,77.7 to 92.9,146.0,None
67,≥1 Dose MenACWY,≥1 Dose,States/Local Areas,New York,2018-2022,Urbanicity,Living In a MSA Non-Principal City,96.0,94.2 to 97.2,1005.0,None


In [100]:
urban_dim = df_urban['Dimension'].unique()
urban_dim

array(['Living In a MSA Principal City',
       'Living In a MSA Non-Principal City', 'Living In a Non-MSA'],
      dtype=object)

In [101]:
# form into a dataframe
urban_dim = pd.DataFrame(urban_dim)
urban_dim.columns = ['Urbanicity']
urban_dim = urban_dim.reset_index(drop=True)
display(urban_dim)

,Urbanicity
0,Living In a MSA Principal City
1,Living In a MSA Non-Principal City
2,Living In a Non-MSA


## Inserting into dimension tables

### Delta Load: Inserting into dose dimension SCD0


In [102]:
try:

    cursor.execute("""
    SELECT setval(
      'dose_dim_dose_id_seq',
      (SELECT MAX(dose_id) FROM dose_dim)
    );
    """)
    conn.commit()

    insert_query = '''
    INSERT INTO dose_dim (dose)
    VALUES (%s) 
    ON CONFLICT (dose) DO NOTHING;
    '''
    for index, row in dose_dim.iterrows():
        print(row['Dose'])
        cursor.execute(insert_query, (row['Dose'],))
    conn.commit()
    print(f"{len(dose_dim)} records inserted into dose_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

≥1 Dose
≥1 Dose Tdap
≥1 Dose Td or Tdap
Up-to-Date
≥3 Dose
≥2 Dose
≥2 Doses or history of disease
≥2 Doses with no disease history
≥1 Dose with no disease history
History of disease
≥2 Doses
≥3 Doses
Series Completion (3 Dose) Among HPV Vaccination Initiators
13 records inserted into dose_dim.


### Delta Load: Inserting into geography dimension SCD1

In [124]:
try:
    insert_query = '''
    INSERT INTO geography_dim (
        state_territory, 
        municipality, 
        county, 
        region, 
        original_geography_type, 
        original_geography
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (original_geography) 
    DO UPDATE SET
        state_territory = EXCLUDED.state_territory,
        municipality = EXCLUDED.municipality,
        county = EXCLUDED.county,
        region = EXCLUDED.region,
        original_geography_type = EXCLUDED.original_geography_type;
    '''
    for index, row in adolescent_geography_dim.iterrows():
        print(row['Geography'])
        cursor.execute(insert_query, (row['state_territory'], row['municipality'], row['county'], row['region'], row['Geography Type'], row['Geography'],))
    conn.commit()
    print(f"{len(adolescent_geography_dim)} records inserted into geography_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Region 7
Region 1
Region 8
Region 10
Region 6
Region 5
Region 9
Region 4
Region 3
Region 2
United States
New York
NY-City of New York
New Jersey
Mississippi
Kentucky
Maine
New Hampshire
Colorado
Nebraska
IL-Rest of state
IL-City of Chicago
PA-Philadelphia
PA-Rest of state
Hawaii
U.S. Virgin Islands
TX-Rest of state
Minnesota
Florida
Oklahoma
Nevada
Montana
Tennessee
Missouri
Washington
Guam
Kansas
Alabama
Utah
California
Alaska
Delaware
Texas
Idaho
Virginia
Iowa
Wyoming
Arkansas
Georgia
South Dakota
Connecticut
Oregon
Rhode Island
New Mexico
Michigan
Louisiana
West Virginia
Massachusetts
District of Columbia
TX-City of Houston
TX-Bexar County
Pennsylvania
North Carolina
Puerto Rico
Maryland
NY-Rest of state
North Dakota
TX-Dallas County
TX-El Paso County
TX-Tarrant County
TX-Hidalgo County
TX-Travis County
Wisconsin
Illinois
Ohio
Arizona
South Carolina
Indiana
Vermont
79 records inserted into geography_dim.


### Delta load: Inserting into race_ethnicity dimension SCD0

In [104]:
try:
    insert_query = '''
    INSERT INTO race_ethnicity_dim (
        race_ethnicity
    )
    VALUES (%s)
    ON CONFLICT (race_ethnicity) 
    DO UPDATE NOTHING;
    '''
    for index, row in race_dim.iterrows():
        print(row['Race and Ethnicity'])
        cursor.execute(insert_query, (row['Race and Ethnicity'],))
    conn.commit()
    print(f"{len(race_dim)} records inserted into race_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

White, Non-Hispanic
Black, Non-Hispanic
Other or Multiple Races, Non-Hispanic
Hispanic
4 records inserted into race_dim.


### Delta load urbanicity dimension SCD0

In [105]:
try:
    insert_query = '''
    INSERT INTO urbanicity_dim (
        urbanicity
    )
    VALUES (%s)
    ON CONFLICT (urbanicity) 
    DO NOTHING;
    '''
    for index, row in urban_dim.iterrows():
        print(row['Urbanicity'])
        cursor.execute(insert_query, (row['Urbanicity'],))
    conn.commit()
    print(f"{len(urban_dim)} records inserted into urbanicity_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Living In a MSA Principal City
Living In a MSA Non-Principal City
Living In a Non-MSA
3 records inserted into urbanicity_dim.


### Delta load insurance dimension SCD2

In [106]:
try:
    insert_query = '''
    INSERT INTO insurance_dim (
        insurance_coverage
    )
    VALUES (%s)
    ON CONFLICT (insurance_coverage) 
    DO UPDATE SET
        insurance_coverage = EXCLUDED.insurance_coverage;
    '''
    for index, row in insurance_dim.iterrows():
        print(row['Insurance Coverage'])
        cursor.execute(insert_query, (row['Insurance Coverage'],))
    conn.commit()
    print(f"{len(insurance_dim)} records inserted into insurance_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Uninsured
Other
Any Medicaid
Private Insurance Only
4 records inserted into insurance_dim.


### Delta Load: Poverty dimension SCD3

In [110]:
poverty_dim

,Poverty,current_poverty_description
0,Below Poverty Level,Living below poverty level. Undefined percentage.
1,Living At or Above Poverty Level,Living above poverty level. Undefined percentage.


In [111]:
try:

    cursor.execute("""
    SELECT setval(
      'poverty_dim_poverty_id_seq',
      (SELECT MAX(poverty_id) FROM poverty_dim)
    );
    """)
    conn.commit()
    insert_query = '''
    INSERT INTO poverty_dim (
        poverty_status,
        current_poverty_description,
        previous_poverty_description
    )
    VALUES (%s, %s, %s)
    ON CONFLICT (poverty_status)
    DO UPDATE SET
        previous_poverty_description = poverty_dim.current_poverty_description,
        current_poverty_description = EXCLUDED.current_poverty_description;
    '''
    for index, row in poverty_dim.iterrows():
        print(f"Processing poverty status: {row['Poverty']}")
        cursor.execute(
            insert_query, 
            (
                row['Poverty'], 
                row['current_poverty_description'], 
                row.get('Previous Poverty Description', None)  # if available or pass None
            )
        )
    conn.commit()
    print(f"{len(poverty_dim)} records inserted into poverty_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()

Processing poverty status: Below Poverty Level
Processing poverty status: Living At or Above Poverty Level
2 records inserted into poverty_dim.
